# ETL PROCESSING

In [1]:
# Import dependencies
import pandas as pd
import re
import json
import pymongo
import geopandas as gpd
import matplotlib.pyplot as plt
import descartes
import os

## Extract

In [2]:
# Read data
data = pd.read_csv("data/Censo_2020.csv")

C:\Users\Carlo\anaconda3\envs\newEnv\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
data.head()

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,LONGITUD,LATITUD,ALTITUD,POBTOT,...,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCINT,VPH_SINTIC,TAMLOC
0,0,Total nacional,0,Total nacional,0,Total nacional,NaN,NaN,NaN,126014024,...,30775898,18307193,15211306,6616141,4047100,1788552,3170894,15108204,852871,*
1,0,Total nacional,0,Total nacional,9998,Localidades de una vivienda,NaN,NaN,NaN,250354,...,47005,8385,18981,1732,1113,12775,14143,51293,7154,*
2,0,Total nacional,0,Total nacional,9999,Localidades de dos viviendas,NaN,NaN,NaN,147125,...,25581,5027,11306,971,708,8247,10065,29741,5283,*
3,1,Aguascalientes,0,Total de la entidad Aguascalientes,0,Total de la Entidad,NaN,NaN,NaN,1425607,...,359895,236003,174089,98724,70126,6021,15323,128996,1711,*
4,1,Aguascalientes,0,Total de la entidad Aguascalientes,9998,Localidades de una vivienda,NaN,NaN,NaN,3697,...,732,205,212,48,41,39,62,530,20,*


In [4]:
data.describe()

,ENTIDAD,MUN,LOC,POBTOT,VIVTOT,TVIVHAB
count,195662.000000,195662.000000,195662.000000,1.956620e+05,1.956620e+05,1.956620e+05
mean,16.806743,59.699400,623.572221,2.582252e+03,9.000329e+02,7.218171e+02
std,8.940159,81.370934,1502.609425,2.931486e+05,1.019782e+05,8.195072e+04
min,0.000000,0.000000,0.000000,1.000000e+00,0.000000e+00,0.000000e+00
25%,8.000000,12.000000,45.000000,4.000000e+00,2.000000e+00,1.000000e+00
50%,16.000000,33.000000,162.000000,1.600000e+01,7.000000e+00,4.000000e+00
75%,25.000000,73.000000,531.000000,1.470000e+02,5.400000e+01,3.800000e+01
max,32.000000,570.000000,9999.000000,1.260140e+08,4.390344e+07,3.523346e+07


In [5]:
data.shape

(195662, 231)

## Transform

In [6]:
# First, we remove all the blank values from rows that sum up total, since they don't have any latitude or longitude we can use
data = data.dropna()

In [7]:
# Then, we create a function to change the coordinates to the correct type
def dms2dd(s):
    # example: s = """0°51'56.29"S"""
    degrees, minutes, seconds, direction = re.split('[°\'"]+', s)
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    if direction in ('S','W'):
        dd*= -1
    return dd


In [8]:
# We use the function to change both latitude and longitude columns
data['LATITUD'] = data['LATITUD'].apply(dms2dd)
data['LONGITUD'] = data['LONGITUD'].apply(dms2dd)*-1


In [9]:
# Here we print all the columns to see which ones will be left on the final Dataframe
for x in data:
    print(x)

ENTIDAD
NOM_ENT
MUN
NOM_MUN
LOC
NOM_LOC
LONGITUD
LATITUD
ALTITUD
POBTOT
POBFEM
POBMAS
P_0A2
P_0A2_F
P_0A2_M
P_3YMAS
P_3YMAS_F
P_3YMAS_M
P_5YMAS
P_5YMAS_F
P_5YMAS_M
P_12YMAS
P_12YMAS_F
P_12YMAS_M
P_15YMAS
P_15YMAS_F
P_15YMAS_M
P_18YMAS
P_18YMAS_F
P_18YMAS_M
P_3A5
P_3A5_F
P_3A5_M
P_6A11
P_6A11_F
P_6A11_M
P_8A14
P_8A14_F
P_8A14_M
P_12A14
P_12A14_F
P_12A14_M
P_15A17
P_15A17_F
P_15A17_M
P_18A24
P_18A24_F
P_18A24_M
P_15A49_F
P_60YMAS
P_60YMAS_F
P_60YMAS_M
REL_H_M
POB0_14
POB15_64
POB65_MAS
PROM_HNV
PNACENT
PNACENT_F
PNACENT_M
PNACOE
PNACOE_F
PNACOE_M
PRES2015
PRES2015_F
PRES2015_M
PRESOE15
PRESOE15_F
PRESOE15_M
P3YM_HLI
P3YM_HLI_F
P3YM_HLI_M
P3HLINHE
P3HLINHE_F
P3HLINHE_M
P3HLI_HE
P3HLI_HE_F
P3HLI_HE_M
P5_HLI
P5_HLI_NHE
P5_HLI_HE
PHOG_IND
POB_AFRO
POB_AFRO_F
POB_AFRO_M
PCON_DISC
PCDISC_MOT
PCDISC_VIS
PCDISC_LENG
PCDISC_AUD
PCDISC_MOT2
PCDISC_MEN
PCON_LIMI
PCLIM_CSB
PCLIM_VIS
PCLIM_HACO
PCLIM_OAUD
PCLIM_MOT2
PCLIM_RE_CO
PCLIM_PMEN
PSIND_LIM
P3A5_NOA
P3A5_NOA_F
P3A5_NOA_M
P6A11_NOA
P6A11_NOAF


In [10]:
# First, we create a dataframe only with the columns we need
final_df = data[["ENTIDAD","NOM_ENT", "MUN", "NOM_MUN","NOM_LOC","LONGITUD","LATITUD","POBTOT","POBFEM","POBMAS","PEA","TVIVPARHAB","VPH_RADIO","VPH_TV","VPH_PC","VPH_TELEF","VPH_CEL","VPH_INTER","VPH_STVP","VPH_SPMVPI","VPH_CVJ"]]
# Then, we replace the asterisk values with zeros and "N/D" values we found when exploring data
final_df = final_df.replace("*",0)
final_df = final_df.replace("N/D",0)

In [11]:
final_df.head()

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,NOM_LOC,LONGITUD,LATITUD,POBTOT,POBFEM,POBMAS,...,TVIVPARHAB,VPH_RADIO,VPH_TV,VPH_PC,VPH_TELEF,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ
7,1,Aguascalientes,1,Aguascalientes,Aguascalientes,-102.296047,21.879823,863893,444725,419168,...,246116,204535,237274,130384,112002,232793,169675,123670,77719,53589
8,1,Aguascalientes,1,Aguascalientes,Granja Adelita,-102.373531,21.871875,5,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,Aguascalientes,1,Aguascalientes,Agua Azul,-102.357122,21.883756,41,17,24,...,12,11,11,5,2,11,4,5,2,1
10,1,Aguascalientes,1,Aguascalientes,Los Arbolitos [Rancho],-102.357295,21.780181,8,0,0,...,0,0,0,0,0,0,0,0,0,0
11,1,Aguascalientes,1,Aguascalientes,Ardillas de Abajo (Las Ardillas),-102.191921,21.945068,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# We realized the columns were "object" type because of the asterisk and N/D values we removed before
final_df.dtypes

ENTIDAD         int64
NOM_ENT        object
MUN             int64
NOM_MUN        object
NOM_LOC        object
LONGITUD      float64
LATITUD       float64
POBTOT          int64
POBFEM         object
POBMAS         object
PEA            object
TVIVPARHAB     object
VPH_RADIO      object
VPH_TV         object
VPH_PC         object
VPH_TELEF      object
VPH_CEL        object
VPH_INTER      object
VPH_STVP       object
VPH_SPMVPI     object
VPH_CVJ        object
dtype: object

In [13]:
# So here we convert the columns to integer type
final_df[final_df.columns[7:].to_list()] = final_df[final_df.columns[7:].to_list()].astype(int)

In [14]:
final_df.dtypes

ENTIDAD         int64
NOM_ENT        object
MUN             int64
NOM_MUN        object
NOM_LOC        object
LONGITUD      float64
LATITUD       float64
POBTOT          int32
POBFEM          int32
POBMAS          int32
PEA             int32
TVIVPARHAB      int32
VPH_RADIO       int32
VPH_TV          int32
VPH_PC          int32
VPH_TELEF       int32
VPH_CEL         int32
VPH_INTER       int32
VPH_STVP        int32
VPH_SPMVPI      int32
VPH_CVJ         int32
dtype: object

In [15]:
#Rename columns
final_df = final_df.rename(
    columns={'NOM_ENT':'NOMBRE_ENTIDAD',
            'MUN':'MUNICIPIO',
            'NOM_MUN':'NOMBRE_MUNICIPIO',
            'NOM_LOC':'NOMBRE_LOCALIDAD',
            })

In [16]:
# Removing latitude and longitude from dataframe and grouping by municipality so we can send it to database
municipios_df = final_df.groupby(['ENTIDAD',"NOMBRE_ENTIDAD","MUNICIPIO","NOMBRE_MUNICIPIO"])[final_df.columns[7:].to_list()].apply(lambda x : x.astype(int).sum()).reset_index()
estados_df = final_df.groupby(['ENTIDAD',"NOMBRE_ENTIDAD"])[final_df.columns[5:].to_list()].apply(lambda x : x.astype(int).sum()).reset_index()

In [17]:
final_df

,ENTIDAD,NOMBRE_ENTIDAD,MUNICIPIO,NOMBRE_MUNICIPIO,NOMBRE_LOCALIDAD,LONGITUD,LATITUD,POBTOT,POBFEM,POBMAS,...,TVIVPARHAB,VPH_RADIO,VPH_TV,VPH_PC,VPH_TELEF,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ
7,1,Aguascalientes,1,Aguascalientes,Aguascalientes,-102.296047,21.879823,863893,444725,419168,...,246116,204535,237274,130384,112002,232793,169675,123670,77719,53589
8,1,Aguascalientes,1,Aguascalientes,Granja Adelita,-102.373531,21.871875,5,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,Aguascalientes,1,Aguascalientes,Agua Azul,-102.357122,21.883756,41,17,24,...,12,11,11,5,2,11,4,5,2,1
10,1,Aguascalientes,1,Aguascalientes,Los Arbolitos [Rancho],-102.357295,21.780181,8,0,0,...,0,0,0,0,0,0,0,0,0,0
11,1,Aguascalientes,1,Aguascalientes,Ardillas de Abajo (Las Ardillas),-102.191921,21.945068,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195656,32,Zacatecas,58,Santa María de la Paz,Los Trigos (Mesa de los Trigos),-103.241053,21.497599,89,43,46,...,23,13,18,2,0,19,1,5,0,0
195657,32,Zacatecas,58,Santa María de la Paz,Mesa Grande,-103.414701,21.553438,165,88,77,...,49,29,44,6,29,24,9,13,0,0
195658,32,Zacatecas,58,Santa María de la Paz,San Isidro,-103.337268,21.486425,1,0,0,...,0,0,0,0,0,0,0,0,0,0
195659,32,Zacatecas,58,Santa María de la Paz,San Miguel Tepetitlán,-103.335932,21.504213,97,50,47,...,31,20,26,4,1,21,12,21,1,1


In [18]:
# Adding some math to get an average percentage of digital inclsuion by municipality
final_df["INCLUSION_EST"] = final_df[final_df.columns[9:].to_list()].sum(axis=1) / 9 / final_df["TVIVPARHAB"]

In [19]:
# Creating an auxiliar dataframe to have the mean of the inclusion by state and then ranking it
pcg_mun = pd.DataFrame(final_df.groupby("ENTIDAD")["INCLUSION_EST"].mean())
pcg_mun["RANKING"] = pcg_mun["INCLUSION_EST"].rank(method="min",ascending=False)
pcg_mun

,INCLUSION_EST,RANKING
ENTIDAD,,
1,0.946187,20.0
2,inf,1.0
3,inf,1.0
4,0.772088,30.0
5,inf,1.0
6,0.852882,23.0
7,inf,1.0
8,0.691562,32.0
9,0.958691,19.0


In [20]:
# Merging both columns from auxiliar dataframe into original DF
estados_df = estados_df.merge(pcg_mun, on="ENTIDAD", how="inner")


In [21]:
estados_df

,ENTIDAD,NOMBRE_ENTIDAD,LONGITUD,LATITUD,POBTOT,POBFEM,POBMAS,PEA,TVIVPARHAB,VPH_RADIO,VPH_TV,VPH_PC,VPH_TELEF,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,INCLUSION_EST,RANKING
0,1,Aguascalientes,-206082,43391,1425603,726401,692484,703101,385119,310947,369261,176868,147738,358691,235652,173721,98641,70047,0.946187,20.0
1,2,Baja California,-637359,173878,3768921,1862302,1891787,2003226,1144174,737594,1067727,575375,576368,1076040,798971,616109,383600,216559,inf,1.0
2,3,Baja California Sur,-280906,61761,798444,389564,401137,422424,238084,141587,212745,108734,91561,224661,148439,135410,67899,36161,inf,1.0
3,4,Campeche,-249148,50230,928353,468301,452676,458236,258474,135507,224870,87608,68630,216864,113845,150782,38478,17964,0.772088,30.0
4,5,Coahuila de Zaragoza,-406796,105908,3146722,1579299,1555882,1479883,897369,640444,861269,367650,371984,821760,519194,442516,195770,123998,inf,1.0
5,6,Colima,-126385,22812,731388,369262,358351,384363,225775,154448,206050,90083,80202,205800,132185,113794,43841,22669,0.852882,23.0
6,7,Chiapas,-1950921,336560,5543657,2816787,2678842,2476880,1337571,727549,1029373,212437,159034,937513,291443,429854,61206,32405,inf,1.0
7,8,Chihuahua,-1292103,333493,3741735,1874993,1834908,1848578,1137297,790101,1063766,487854,460238,1045252,649365,479293,281732,179737,0.691562,32.0
8,9,Ciudad de México,-62700,12046,9209944,4804711,4404549,5099560,2756126,2165829,2647768,1649355,1898220,2536367,2084106,1290772,957148,568815,0.958691,19.0
9,10,Durango,-615406,141689,1832635,921586,895437,812723,490152,315748,455770,169467,137678,432234,214681,221731,73031,54612,inf,1.0


In [22]:
estados_df.to_csv("data/Censo_por_estado.csv", index = False)

## Transform GEOJSON

In [23]:
# Her we read again the data so we can start this part without doing all the extract part again
data = pd.read_csv("data/Censo_por_estado.csv")

In [ ]:
## Loop for get every GeoJSON from the shape files by municipalities
directory = 'C:/Users/Carlo/OneDrive/Escritorio/Proyecto 2/Estados/'
## for that we use OS module in python that provides functions to interact with the operating system and files.
for filename in os.listdir(directory):
    ## files with extention .shp was elected (very importan you need the five files related with municipalities so as to get the correct info)
    if filename.endswith('.shp'):
        ## read file iterator with geopandas
        mapa = gpd.read_file(filename)
        #change to coordinates EPSG:4326
        mapa = mapa.to_crs('EPSG:4326')
        #change to coordinates EPSG:4326
        mapa.to_file(f'{filename[0:4]}.geojson', driver = 'GeoJSON')
        mapa.plot()
        #save the image just in case we need to put in flat coordinates in our dashboard
        plt.savefig(f'{filename[0:4]}.png')
    else:
        continue

In [ ]:
## Loop for make an append and have a unique GeoJson with all the shapes
directory = 'C:/Users/Carlo/OneDrive/Escritorio/Proyecto 2/Estados/'
## for that we use OS module in python that provides functions to interact with the operating system and files.
for filename in os.listdir(directory):
    ## files with extention .shp was elected (very importan you need the five files related with municipalities so as to get the correct info)
    if filename.endswith('.geojson'):
        ## read file iterator with geopandas
        shapemun = gpd.read_file()
        #change to coordinates EPSG:4326
        mapa = mapa.to_crs('EPSG:4326')
        #change to coordinates EPSG:4326
        mapa.to_file(f'{filename[0:4]}.geojson', driver = 'GeoJSON')
        mapa.plot()
        #save the image just in case we need to put in flat coordinates in our dashboard
        plt.savefig(f'{filename[0:4]}.png')
    else:
        continue

In [24]:
# Loop for getting a list of reading geojson files so as to use in a concat function
directory = 'C:/Users/Carlo/OneDrive/Escritorio/Proyecto 2/Estados/'
geojson_mun = []
for file in os.listdir(directory):
    if file.endswith('geojson'):
        df = gpd.read_file(f'{directory}{file}')
        geojson_mun.append(df)
    else:
        continue


In [25]:
# Concat all the geojson files into one and show as DF
final_geometry = pd.concat(geojson_mun, axis=0, join='outer', ignore_index=False)

In [26]:
# Exploring final geometry for dtype objects
final_geometry.dtypes
# Changing the type of the columns CVE_ENT,CVE_MUN
final_geometry[['CVE_ENT']]=final_geometry[['CVE_ENT']].astype(int)

In [27]:
# Meking the merge with data and shapemun
merge = final_geometry.merge(data,how='left', left_on=['CVE_ENT'],right_on=['ENTIDAD'])
merge

,CVEGEO,CVE_ENT,NOMGEO,geometry,ENTIDAD,NOMBRE_ENTIDAD,LONGITUD,LATITUD,POBTOT,POBFEM,...,VPH_TV,VPH_PC,VPH_TELEF,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,INCLUSION_EST,RANKING
0,01,1,Aguascalientes,"POLYGON ((-102.28787 22.41649, -102.28753 22.4...",1,Aguascalientes,-206082,43391,1425603,726401,...,369261,176868,147738,358691,235652,173721,98641,70047,0.946187,20.0
1,02,2,Baja California,"MULTIPOLYGON (((-112.30067 28.38702, -112.2982...",2,Baja California,-637359,173878,3768921,1862302,...,1067727,575375,576368,1076040,798971,616109,383600,216559,inf,1.0
2,03,3,Baja California Sur,"MULTIPOLYGON (((-109.89451 22.87476, -109.8946...",3,Baja California Sur,-280906,61761,798444,389564,...,212745,108734,91561,224661,148439,135410,67899,36161,inf,1.0
3,04,4,Campeche,"MULTIPOLYGON (((-91.95650 20.19700, -91.95636 ...",4,Campeche,-249148,50230,928353,468301,...,224870,87608,68630,216864,113845,150782,38478,17964,0.772088,30.0
4,05,5,Coahuila de Zaragoza,"POLYGON ((-102.31079 29.87695, -102.31050 29.8...",5,Coahuila de Zaragoza,-406796,105908,3146722,1579299,...,861269,367650,371984,821760,519194,442516,195770,123998,inf,1.0
5,06,6,Colima,"MULTIPOLYGON (((-114.73734 18.37206, -114.7364...",6,Colima,-126385,22812,731388,369262,...,206050,90083,80202,205800,132185,113794,43841,22669,0.852882,23.0
6,07,7,Chiapas,"POLYGON ((-91.97364 17.91143, -91.96993 17.910...",7,Chiapas,-1950921,336560,5543657,2816787,...,1029373,212437,159034,937513,291443,429854,61206,32405,inf,1.0
7,08,8,Chihuahua,"POLYGON ((-107.65314 31.78369, -107.65314 31.7...",8,Chihuahua,-1292103,333493,3741735,1874993,...,1063766,487854,460238,1045252,649365,479293,281732,179737,0.691562,32.0
8,09,9,Ciudad de México,"POLYGON ((-99.11124 19.56150, -99.11485 19.557...",9,Ciudad de México,-62700,12046,9209944,4804711,...,2647768,1649355,1898220,2536367,2084106,1290772,957148,568815,0.958691,19.0
9,10,10,Durango,"POLYGON ((-105.96732 26.78554, -105.93919 26.7...",10,Durango,-615406,141689,1832635,921586,...,455770,169467,137678,432234,214681,221731,73031,54612,inf,1.0


In [28]:
# Drop repeated columns
merge = merge.drop(columns=['CVEGEO','CVE_ENT','NOMGEO'])
merge

,geometry,ENTIDAD,NOMBRE_ENTIDAD,LONGITUD,LATITUD,POBTOT,POBFEM,POBMAS,PEA,TVIVPARHAB,...,VPH_TV,VPH_PC,VPH_TELEF,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,INCLUSION_EST,RANKING
0,"POLYGON ((-102.28787 22.41649, -102.28753 22.4...",1,Aguascalientes,-206082,43391,1425603,726401,692484,703101,385119,...,369261,176868,147738,358691,235652,173721,98641,70047,0.946187,20.0
1,"MULTIPOLYGON (((-112.30067 28.38702, -112.2982...",2,Baja California,-637359,173878,3768921,1862302,1891787,2003226,1144174,...,1067727,575375,576368,1076040,798971,616109,383600,216559,inf,1.0
2,"MULTIPOLYGON (((-109.89451 22.87476, -109.8946...",3,Baja California Sur,-280906,61761,798444,389564,401137,422424,238084,...,212745,108734,91561,224661,148439,135410,67899,36161,inf,1.0
3,"MULTIPOLYGON (((-91.95650 20.19700, -91.95636 ...",4,Campeche,-249148,50230,928353,468301,452676,458236,258474,...,224870,87608,68630,216864,113845,150782,38478,17964,0.772088,30.0
4,"POLYGON ((-102.31079 29.87695, -102.31050 29.8...",5,Coahuila de Zaragoza,-406796,105908,3146722,1579299,1555882,1479883,897369,...,861269,367650,371984,821760,519194,442516,195770,123998,inf,1.0
5,"MULTIPOLYGON (((-114.73734 18.37206, -114.7364...",6,Colima,-126385,22812,731388,369262,358351,384363,225775,...,206050,90083,80202,205800,132185,113794,43841,22669,0.852882,23.0
6,"POLYGON ((-91.97364 17.91143, -91.96993 17.910...",7,Chiapas,-1950921,336560,5543657,2816787,2678842,2476880,1337571,...,1029373,212437,159034,937513,291443,429854,61206,32405,inf,1.0
7,"POLYGON ((-107.65314 31.78369, -107.65314 31.7...",8,Chihuahua,-1292103,333493,3741735,1874993,1834908,1848578,1137297,...,1063766,487854,460238,1045252,649365,479293,281732,179737,0.691562,32.0
8,"POLYGON ((-99.11124 19.56150, -99.11485 19.557...",9,Ciudad de México,-62700,12046,9209944,4804711,4404549,5099560,2756126,...,2647768,1649355,1898220,2536367,2084106,1290772,957148,568815,0.958691,19.0
9,"POLYGON ((-105.96732 26.78554, -105.93919 26.7...",10,Durango,-615406,141689,1832635,921586,895437,812723,490152,...,455770,169467,137678,432234,214681,221731,73031,54612,inf,1.0


In [30]:
# Move geometry to the end of the data_frame
merge.columns
merge = merge[['ENTIDAD', 'NOMBRE_ENTIDAD', 'POBTOT', 'POBFEM',
       'POBMAS', 'PEA', 'TVIVPARHAB', 'VPH_RADIO', 'VPH_TV', 'VPH_PC',
       'VPH_TELEF', 'VPH_CEL', 'VPH_INTER', 'VPH_STVP', 'VPH_SPMVPI',
       'VPH_CVJ', 'INCLUSION_EST','RANKING', 'geometry']]
merge.head()

,ENTIDAD,NOMBRE_ENTIDAD,POBTOT,POBFEM,POBMAS,PEA,TVIVPARHAB,VPH_RADIO,VPH_TV,VPH_PC,VPH_TELEF,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,INCLUSION_EST,RANKING,geometry
0,1,Aguascalientes,1425603,726401,692484,703101,385119,310947,369261,176868,147738,358691,235652,173721,98641,70047,0.946187,20.0,"POLYGON ((-102.28787 22.41649, -102.28753 22.4..."
1,2,Baja California,3768921,1862302,1891787,2003226,1144174,737594,1067727,575375,576368,1076040,798971,616109,383600,216559,inf,1.0,"MULTIPOLYGON (((-112.30067 28.38702, -112.2982..."
2,3,Baja California Sur,798444,389564,401137,422424,238084,141587,212745,108734,91561,224661,148439,135410,67899,36161,inf,1.0,"MULTIPOLYGON (((-109.89451 22.87476, -109.8946..."
3,4,Campeche,928353,468301,452676,458236,258474,135507,224870,87608,68630,216864,113845,150782,38478,17964,0.772088,30.0,"MULTIPOLYGON (((-91.95650 20.19700, -91.95636 ..."
4,5,Coahuila de Zaragoza,3146722,1579299,1555882,1479883,897369,640444,861269,367650,371984,821760,519194,442516,195770,123998,inf,1.0,"POLYGON ((-102.31079 29.87695, -102.31050 29.8..."


## Load

In [31]:
# Create a connection to mongo, create a database and a collection for municipalities
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)
# Select database and collection to use
db = client.inclusion_digital
estados = db.estados


In [32]:
# Loop for every register on the DataFrame to save each one as a document
for index,row in merge.iterrows():
    merge_partial = merge[index:index + 1]
    resultado_json = (
        merge_partial
        .sort_values(by="ENTIDAD")
        .reset_index()
        .loc[:,merge_partial.columns.to_list()]
        .to_json())
    resultado_json = json.loads(resultado_json)
    estados.insert_one(resultado_json)


In [33]:
# save a copy as a json
final_json = (
    merge
    .sort_values(by="ENTIDAD")
    .reset_index()
    .loc[:,merge.columns.to_list()]
    .to_json()
)
with open("C:/Users/Carlo/OneDrive/Documents/project2/data/estados.json", "w") as outfile: 
    json.dump(final_json, outfile)